In [1]:
# import the needed libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model

# config
img_width, img_height = 50,50 #width & height of input image
input_depth = 1 #1: gray image
train_data_dir = 'D:/Datasets/testchairs/training' #data training path
testing_data_dir = 'D:/Datasets/testchairs/testing' #data testing path
epochs = 15 #number of training epoch
batch_size = 5 #training batch size

# define image generator for Keras,
# here, we map pixel intensity to 0-1 /
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# read image batch by batch
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',#inpput iameg: gray
    target_size=(img_width,img_height),#input image size
    batch_size=batch_size,#batch size
    shuffle=True,#randomise images order
    class_mode='categorical')#categorical: one-hot encoding format class label
testing_generator = test_datagen.flow_from_directory(
    testing_data_dir,
    color_mode='grayscale',
    target_size=(img_width,img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical')

print(train_generator.class_indices)
print(testing_generator.class_indices)

Found 1252 images belonging to 5 classes.
Found 12 images belonging to 5 classes.
{'Comedor': 0, 'Gamer': 1, 'Mesedoras': 2, 'Taburete': 3, 'Terraza': 4}
{'Comedor': 0, 'Gamer': 1, 'Mesedoras': 2, 'Taburete': 3, 'Terraza': 4}


In [2]:
# define number of filters and nodes in the fully connected layer
NUMB_FILTER_L1 = 64
NUMB_FILTER_L2 = 64
NUMB_FILTER_L3 = 128
NUMB_NODE_FC_LAYER = 10

#define input image order shape
if K.image_data_format() == 'channels_first':
    input_shape_val = (input_depth, img_width, img_height)
else:
    input_shape_val = (img_width, img_height, input_depth)

#define the network
model = Sequential()

# Layer 1
model.add(Conv2D(NUMB_FILTER_L1, (5, 5), 
                 input_shape=input_shape_val, 
                 padding='same', name='input_tensor'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Layer 2
model.add(Conv2D(NUMB_FILTER_L2, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Layer 3
model.add(Conv2D(NUMB_FILTER_L3, (5, 5), padding='same'))
model.add(Activation('relu'))

# flattening the model for fully connected layer
model.add(Flatten())

# fully connected layer
model.add(Dense(NUMB_NODE_FC_LAYER, activation='relu'))

# output layer
model.add(Dense(train_generator.num_classes, 
                activation='softmax', name='output_tensor'))

# Compilile the network
model.compile(loss='categorical_crossentropy',
              optimizer='sgd', metrics=['accuracy'])

# Show the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (Conv2D)       (None, 50, 50, 64)        1664      
                                                                 
 activation (Activation)     (None, 50, 50, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 64)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 25, 25, 64)        102464    
                                                                 
 activation_1 (Activation)   (None, 25, 25, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                    

In [3]:
# Train and test the network
model.fit(
    train_generator,#our training generator
    #number of iteration per epoch = number of data / batch size
    steps_per_epoch=np.floor(train_generator.n/batch_size),
    epochs=epochs,#number of epoch
    validation_data=testing_generator,#our validation generator
    #number of iteration per epoch = number of data / batch size
    validation_steps=np.floor(testing_generator.n / batch_size))

Epoch 1/15
250/250 [==============================] - 9s 34ms/step - loss: 1.0919 - accuracy: 0.4739 - val_loss: 3.5519 - val_accuracy: 0.3000
Epoch 2/15
250/250 [==============================] - 8s 31ms/step - loss: 0.7124 - accuracy: 0.6889 - val_loss: 3.1804 - val_accuracy: 0.4000
Epoch 3/15
250/250 [==============================] - 8s 34ms/step - loss: 0.5503 - accuracy: 0.7626 - val_loss: 3.9793 - val_accuracy: 0.4000
Epoch 4/15
250/250 [==============================] - 8s 31ms/step - loss: 0.4463 - accuracy: 0.8132 - val_loss: 4.0762 - val_accuracy: 0.2000
Epoch 5/15
250/250 [==============================] - 8s 30ms/step - loss: 0.3882 - accuracy: 0.8268 - val_loss: 5.8079 - val_accuracy: 0.2000
Epoch 6/15
250/250 [==============================] - 8s 30ms/step - loss: 0.3737 - accuracy: 0.8500 - val_loss: 5.3471 - val_accuracy: 0.3000
Epoch 7/15
250/250 [==============================] - 7s 30ms/step - loss: 0.3320 - accuracy: 0.8589 - val_loss: 4.7011 - val_accuracy: 0.3000

In [4]:
print("Training is done!")
model.save('./model/modelLeNet5.h5')
print("Model is successfully stored!")

Training is done!
Model is successfully stored!
